## 딥러닝 2주차

### 인공신경망

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# 데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# MNIST 데이터셋 로드
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1133)>



100%|██████████| 9912422/9912422 [00:01<00:00, 5882948.83it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1133)>



100%|██████████| 28881/28881 [00:00<00:00, 147598.84it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1133)>



100%|██████████| 1648877/1648877 [00:00<00:00, 1773682.51it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1133)>



100%|██████████| 4542/4542 [00:00<00:00, 2262533.11it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [ ]:
import matplotlib.pyplot as plt

img, label = next(iter(trainloader))

plt.imshow(img[0, 0, :, :], 'gray')
plt.title(f'{label[0]}')
plt.show()


In [3]:
class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 입력층에서 은닉층으로
        self.fc2 = nn.Linear(128, 64)       # 은닉층에서 은닉층으로
        self.fc3 = nn.Linear(64, 10)        # 은닉층에서 출력층으로

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 입력 이미지를 1차원 벡터로 변환
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
# 모델 초기화
model = SimpleANN()

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 모델 학습
for epoch in range(10):  # 10 에포크 동안 학습
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 기울기 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 출력
        running_loss += loss.item()
        if i % 100 == 99:  # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 1.369
[Epoch 1, Batch 200] loss: 0.486
[Epoch 1, Batch 300] loss: 0.408
[Epoch 1, Batch 400] loss: 0.353
[Epoch 1, Batch 500] loss: 0.270
[Epoch 1, Batch 600] loss: 0.310
[Epoch 1, Batch 700] loss: 0.262
[Epoch 1, Batch 800] loss: 0.227
[Epoch 1, Batch 900] loss: 0.231
[Epoch 2, Batch 100] loss: 0.186
[Epoch 2, Batch 200] loss: 0.177
[Epoch 2, Batch 300] loss: 0.183
[Epoch 2, Batch 400] loss: 0.180
[Epoch 2, Batch 500] loss: 0.205
[Epoch 2, Batch 600] loss: 0.164
[Epoch 2, Batch 700] loss: 0.166
[Epoch 2, Batch 800] loss: 0.154
[Epoch 2, Batch 900] loss: 0.162
[Epoch 3, Batch 100] loss: 0.156
[Epoch 3, Batch 200] loss: 0.132
[Epoch 3, Batch 300] loss: 0.136
[Epoch 3, Batch 400] loss: 0.132
[Epoch 3, Batch 500] loss: 0.125
[Epoch 3, Batch 600] loss: 0.139
[Epoch 3, Batch 700] loss: 0.125
[Epoch 3, Batch 800] loss: 0.121
[Epoch 3, Batch 900] loss: 0.112
[Epoch 4, Batch 100] loss: 0.106
[Epoch 4, Batch 200] loss: 0.118
[Epoch 4, Batch 300] loss: 0.111
[Epoch 4, 

In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10000 test images: 97.59%
